In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

In [2]:
df = pd.read_csv("AugmentedExercises.csv")
df.head()

,Title,Desc,Type,BodyPart,Equipment,Level,Rating,Movement,MovementType,Intensity,InjuryRisk,SecondaryMuscles
0,Barbell roll-out,The barbell roll-out is an abdominal exercise ...,Strength,Abdominals,Barbell,Intermediate,8.9,core,Isolation,High,Low,NaN
1,Barbell Ab Rollout - On Knees,The barbell roll-out is an abdominal exercise ...,Strength,Abdominals,Barbell,Intermediate,8.9,core,Isolation,High,Low,NaN
2,Dumbbell V-Sit Cross Jab,The dumbbell V-sit cross jab is a hybrid movem...,Strength,Abdominals,Dumbbell,Intermediate,9.3,core,Isolation,Medium,Low,NaN
3,Dumbbell spell caster,The dumbbell spell caster is an exercise that ...,Strength,Abdominals,Dumbbell,Beginner,9.3,core,Compound,Medium,Low,Obliques
4,Landmine twist,The landmine twist is a rotational abdominal m...,Strength,Abdominals,Other,Intermediate,9.5,core,Compound,Low,Low,Obliques


In [3]:
# Split SecondaryMuscles into individual terms
df['SecondaryMuscles'] = df['SecondaryMuscles'].fillna('').str.split(', ')

# Create binary matrices for categorical features
bodypart_dummies = pd.get_dummies(df['BodyPart'], prefix='BodyPart')
movement_dummies = pd.get_dummies(df['Movement'], prefix='Movement')
equipment_dummies = pd.get_dummies(df['Equipment'], prefix='Equipment')
level_dummies = pd.get_dummies(df['Level'], prefix='Level')

# Explode and encode SecondaryMuscles
secondary_exp = df['SecondaryMuscles'].explode()
secondary_dummies = pd.get_dummies(secondary_exp, prefix='Secondary').groupby(level=0).max()

# Combine all categorical features
categorical_matrix = pd.concat([
    bodypart_dummies, 
    movement_dummies, 
    equipment_dummies,
    level_dummies,
    secondary_dummies
], axis=1).fillna(0)

In [4]:
categorical_sim = cosine_similarity(categorical_matrix)

In [5]:
tfidf = TfidfVectorizer(stop_words='english')
desc_matrix = tfidf.fit_transform(df['Desc'])
text_sim = linear_kernel(desc_matrix, desc_matrix)
combined_sim = 0.45 * categorical_sim + 0.55 * text_sim

In [6]:
indices = pd.Series(df.index, index=df['Title']).drop_duplicates()

In [7]:
def get_recommendations(title, similarity_matrix=combined_sim, num_recommend=10):
    idx = indices[title]
    sim_scores = list(enumerate(similarity_matrix[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    top_indices = [i[0] for i in sim_scores[1:num_recommend+1]]
    return df.iloc[top_indices]

In [8]:
get_recommendations('Close-Grip Front Lat Pulldown', num_recommend = 10)

,Title,Desc,Type,BodyPart,Equipment,Level,Rating,Movement,MovementType,Intensity,InjuryRisk,SecondaryMuscles
98,Close-grip pull-down,The close-grip pull-down is a variation on the...,Strength,Lats,Cable,Intermediate,8.9,pull,Isolation,Low,Low,[]
101,Straight-arm rope pull-down,The straight-arm rope pull-down is a variation...,Strength,Lats,Cable,Intermediate,8.7,pull,Isolation,Low,Low,[]
99,Shotgun row,The shotgun row is a single-arm row variation ...,Strength,Lats,Cable,Intermediate,8.9,pull,Isolation,Low,Low,[]
97,Weighted pull-up,The weighted pull-up is a more advanced variat...,Strength,Lats,Other,Intermediate,9.5,pull,Isolation,Low,Low,[]
102,Pullups,The pull-up is a multi-joint bodyweight exerci...,Strength,Lats,Body Only,Intermediate,9.2,pull,Isolation,Low,Low,[]
113,Seated Cable Rows,The cable seated row is a popular exercise to ...,Strength,Middle Back,Cable,Intermediate,8.8,pull,Isolation,Low,Low,[]
115,Seated Cable Rows,The cable seated row is a popular exercise to ...,Strength,Middle Back,Cable,Intermediate,8.8,pull,Isolation,Low,Low,[]
117,Seated Cable Rows,The cable seated row is a popular exercise to ...,Strength,Middle Back,Cable,Intermediate,8.8,pull,Isolation,Low,Low,[]
104,Rope climb,The rope climb is an exercise that is used com...,Strength,Lats,Body Only,Intermediate,8.7,pull,Isolation,Low,Low,[]
46,Overhead cable curl,"Also known as a crucifix curl, the overhead ca...",Strength,Biceps,Cable,Intermediate,9.0,pull,Isolation,Low,Low,[]


In [9]:
df[df["Rating"]==df["Rating"].min()]

,Title,Desc,Type,BodyPart,Equipment,Level,Rating,Movement,MovementType,Intensity,InjuryRisk,SecondaryMuscles
28,Crunches,The crunch is a popular core exercise targetin...,Strength,Abdominals,Body Only,Intermediate,8.7,core,Compound,Low,Low,[Obliques]
42,Dumbbell Alternate Bicep Curl,The alternating biceps curl is a single-joint ...,Strength,Biceps,Dumbbell,Intermediate,8.7,pull,Isolation,Medium,Low,[]
43,Single-arm dumbbell preacher curl,The single-arm dumbbell preacher curl is an ex...,Strength,Biceps,Dumbbell,Intermediate,8.7,pull,Isolation,Medium,Low,[]
44,Alternate Incline Dumbbell Curl,The alternating incline dumbbell biceps curl i...,Strength,Biceps,Dumbbell,Intermediate,8.7,pull,Isolation,Medium,Low,[]
45,Standing concentration curl,The standing concentration curl is a variation...,Strength,Biceps,Dumbbell,Beginner,8.7,pull,Isolation,Medium,Low,[]
52,Seated Calf Raise,The machine seated calf raise is an exercise t...,Strength,Calves,Machine,Intermediate,8.7,legs,Isolation,Low,Low,[]
53,Calf Press On The Leg Press Machine,The leg press calf raise is an exercise that u...,Strength,Calves,Machine,Intermediate,8.7,legs,Isolation,Low,Low,[]
61,Hands-elevated push-up,The hands-elevated push-up is a variation on t...,Strength,Chest,Body Only,Beginner,8.7,push,Compound,Low,Low,[Triceps]
65,Incline cable chest press,The incline cable chest press is a pressing mo...,Strength,Chest,Cable,Beginner,8.7,push,Isolation,Low,Low,[]
72,Neutral-grip dumbbell bench press,The neutral-grip dumbbell bench press is a var...,Strength,Chest,Dumbbell,Intermediate,8.7,push,Compound,Medium,Low,[Triceps]
